In [ ]:
# watermark synthesis
import os 
import random
import shutil
from PIL import Image
import numpy as np

def trans_paste(bg_img,fg_img,mask,box=(0,0)):
    fg_img_trans = Image.new("RGBA",bg_img.size)
    fg_img_trans.paste(fg_img,box,mask=mask)
    new_img = Image.alpha_composite(bg_img,fg_img_trans)
    return new_img,fg_img_trans

if os.path.isdir('dataset'):
    shutil.rmtree('dataset')

os.mkdir('dataset')
BASE_IMG_DIR = '/Users/oishii/Downloads/val2014/'
WATERMARK_DIR = 'logos' #1080 
images = sorted([os.path.join(BASE_IMG_DIR,x) for x in os.listdir(BASE_IMG_DIR) if '.jpg' in x])
watermarks = sorted([os.path.join(WATERMARK_DIR,x).replace(' ','_') for x in os.listdir(WATERMARK_DIR) if '.png' in x])
# rename all the watermark from replace ' ' to '_'

random.shuffle(images)
random.shuffle(watermarks)

train_images = images[:int(len(images)*0.7)]
val_images = images[int(len(images)*0.7):int(len(images)*0.8)]
test_images = images[int(len(images)*0.8):]

train_wms = watermarks[:int(len(watermarks)*0.7)]
val_wms = watermarks[int(len(watermarks)*0.7):int(len(watermarks)*0.8)]
test_wms = watermarks[int(len(watermarks)*0.8):]

# save all the settings to file
names = ['train_images','val_images','test_images','train_wms','val_wms','test_wms']
lists = [train_images,val_images,test_images,train_wms,val_wms,test_wms]
dataset = dict(zip(names, lists))

for name,content in dataset.items():
    with open('dataset/%s.txt'%name,'w') as f:
        f.write("\n".join(content))

print('SAVE ALL THE SETTING')

for name, images in dataset.items():
    if 'images' not in name:
        continue
    # for each setting, synthesis the watermark
    # for each image, add X(X=6) watermark in differnet position, alpha,
    # save the synthesized image, watermark mask, reshaped mask,
    save_path = 'dataset/%s/'%name
    os.makedirs('%s/image'%(save_path))
    os.makedirs('%s/mask'%(save_path))
    os.makedirs('%s/wm'%(save_path))
    
    for img in images:
        im = Image.open(img).convert('RGBA')
        imw,imh = im.size
        
        for wmg in random.choices(dataset[name.replace('images','wms')],k=6):
            wm = Image.open(wmg.replace('_',' ')).convert("RGBA") # RGBA
            # get the mask of wm
            # data agumentation of wm
            wm = wm.rotate(angle=random.randint(0,360),expand=True) # rotate
            
            # make sure the 
            imrw = random.randrange(int(0.4*imw),int(0.8*imw))
            imrh = random.randrange(int(0.4*imh),int(0.8*imh))
            wmsize = imrh if imrw > imrh else imrw
            wm = wm.resize((wmsize,wmsize),Image.BILINEAR)
            w,h = wm.size # new size 
            
            box_left = random.randint(0,imw-w)
            box_upper = random.randint(0,imh-h)
            wmm = wm.copy()
            wm.putalpha(random.randint(int(255*0.4),int(255*0.8))) # alpha
            
            ims,wmc = trans_paste(im,wm,wmm,(box_left,box_upper))
            
            wmnp = np.array(wmc) # h,w,3
            mask = np.sum(wmnp,axis=2)>0
            mm = Image.fromarray(np.uint8(mask*255),mode='L')
            
            identifier = os.path.basename(img).split('.')[0] +'-'+os.path.basename(wmg).split('.')[0] + '.png'
            # save 
            wmc.save('%s/wm/%s'%(save_path,identifier))
            ims.save('%s/image/%s'%(save_path,identifier))
            mm.save('%s/mask/%s'%(save_path,identifier))
            
            

SAVE ALL THE SETTING
